In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
import dask.dataframe as dd
import dask.array as da

In [6]:
from dask.distributed import Client

Zadanie 1

In [7]:
df = pd.read_csv('zamowienia.csv', sep=';')
df.dtypes

Kraj                object
Sprzedawca          object
Data zamowienia     object
idZamowienia         int64
Utarg              float64
dtype: object

In [11]:
ddf = dd.read_csv('zamowienia_missing.csv', delimiter=',')
ddf.dtypes

Unnamed: 0           int64
Kraj                object
Sprzedawca          object
Data zamowienia     object
idZamowienia       float64
Utarg              float64
dtype: object

In [16]:
ddf.head(25)

,Unnamed: 0,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,0,Polska,Kowalski,2003-07-16,10248.0,440.00
1,1,Polska,Sowiński,2003-07-10,10249.0,1863.40
2,2,Niemcy,Peacock,2003-07-12,10250.0,1552.60
3,3,Niemcy,Leverling,2003-07-15,10251.0,654.06
4,4,Niemcy,Peacock,2003-07-11,10252.0,3597.90
5,5,Niemcy,Leverling,2003-07-16,10253.0,1444.80
6,6,Polska,Kowalski,2003-07-23,10254.0,556.62
7,7,Polska,Dudek,2003-07-15,10255.0,2490.50
8,8,Niemcy,Leverling,2003-07-17,10256.0,517.80
9,9,Niemcy,Peacock,2003-07-22,10257.0,1119.90


In [17]:
result = ddf['Utarg'].mean().compute()
result

np.float64(1538.6959899749372)

In [18]:
ddf[ddf['Kraj'] == 'Polska']['Utarg'].sum().compute()

np.float64(332635.29)

In [19]:
ddf_sample = dd.read_csv('zamowienia_missing.csv', sep=',', sample=1000)
ddf_sample[ddf_sample['Kraj'] == 'Polska']['Utarg'].sum().compute()

np.float64(332635.29)

In [20]:
ddf_sample = dd.read_csv('zamowienia_missing.csv', sep=',', sample=70)
ddf_sample[ddf_sample['Kraj'] == 'Polska']['Utarg'].sum().compute()

np.float64(332635.29)

Zadanie 3

In [21]:
client = Client(n_workers=3, memory_limit='16GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 12,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52413,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:52430,Total threads: 4
Dashboard: http://127.0.0.1:52434/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:52416,


In [24]:
start_time = time.time()
ddfp = dd.read_parquet('*.parquet')
end_time = time.time()
print(f"Czas ładowania danych: {end_time - start_time} sekund")

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\io\parquet\core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


Czas ładowania danych: 0.3510286808013916 sekund


Zadanie 4

In [29]:
ddfp = dd.read_parquet('*.parquet')
ddfp = ddfp.persist()
start_time = time.time()
top10 = ddfp.groupby('username')['likes'].sum().nlargest(10).compute()
end_time = time.time()
print(f"Czas: {end_time - start_time} sekund")

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\io\parquet\core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


Czas: 1.240492582321167 sekund


In [30]:
top10

username
instagram          29864166
lizakoshy          19217644
433                16457870
amandacerny        15019135
akshaykumar        13352324
maisie_williams    12808999
chiaraferragni     11709658
claireholt         10109849
danbilzerian        9253425
urvashirautela      9104884
Name: likes, dtype: int64

In [38]:
ddfp = dd.read_parquet('*.parquet')
ddfp = ddfp.persist()
ddfp['date'] = dd.to_datetime(ddfp['date'])
start_time = time.time()
polrocze = ddfp[ddfp['date'].between('2019-01-01', '2019-06-30')].compute()
end_time = time.time()
print(f"Czas: {end_time - start_time} sekund")

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\io\parquet\core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


Czas: 11.955950498580933 sekund


In [39]:
polrocze

,sid,sid_profile,post_id,profile_id,date,post_type,description,likes,comments,username,bio,following,followers,num_posts,is_business_account,lang,category
2,28370905,3496776,Bunhd1DFVAG,2237947779,2019-03-05 08:03:11,1,Tech Tuesday. Been flat out on the tools. Got ...,168,3,andylund_,"Professional Bicycle technician, Intense Racin...",520,1204,494,False,en,science_&_technology
4,32170690,3496776,BuDfIyslzfw,2237947779,2019-02-19 08:10:11,1,Solid effort on the bar turn.\nFully turned.\n...,145,2,andylund_,"Professional Bicycle technician, Intense Racin...",520,1204,494,False,en,diaries_&_daily_life
5,14315358,3496776,BxJsMDpA2yH,2237947779,2019-05-07 08:33:51,1,Annual springtime flora picture.\nTurn bars in...,124,2,andylund_,"Professional Bicycle technician, Intense Racin...",520,1204,494,False,en,arts_&_culture
6,8304346,3496776,Bt5LFpZlm3z,2237947779,2019-02-15 08:02:35,1,Laps in spring like conditions. Getting these ...,150,3,andylund_,"Professional Bicycle technician, Intense Racin...",520,1204,494,False,en,sports
7,14315346,3496776,BxZIzaQhS-o,2237947779,2019-05-13 08:32:30,1,Cheers Scotland 🏴󠁧󠁢󠁳󠁣󠁴󠁿 See you in a few weeks...,166,2,andylund_,"Professional Bicycle technician, Intense Racin...",520,1204,494,False,en,sports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147352,43194859,4492285,BxEr8JbHVIZ,3198207653,2019-05-05 09:55:28,1,A restaurant with no sign and a one-word menu 🍜,112,2,zarazhangg,Harvard grad from China working in tech 🇨🇳🇸🇬🇯🇵...,1801,1428,249,False,en,food_&_dining
1147353,43194861,4492285,Bw6NjgYH9VF,3198207653,2019-05-01 08:17:33,1,Beijing has nice days too,151,3,zarazhangg,Harvard grad from China working in tech 🇨🇳🇸🇬🇯🇵...,1801,1428,249,False,en,diaries_&_daily_life
1147354,43194863,4492285,Bwossrynm05,3198207653,2019-04-24 13:03:22,1,Breaded,149,2,zarazhangg,Harvard grad from China working in tech 🇨🇳🇸🇬🇯🇵...,1801,1428,249,False,en,food_&_dining
1147355,43194867,4492285,Bvx0jgoHk8h,3198207653,2019-04-03 05:33:47,1,A taste of Tokyo in Beijing 🍱 #waimai,74,1,zarazhangg,Harvard grad from China working in tech 🇨🇳🇸🇬🇯🇵...,1801,1428,249,False,en,travel_&_adventure


Zadanie 5

In [40]:
dtype_opt = {
    'sid': np.int32,
    'post_id': 'category',
    'profile_id': 'category',
    'post_type': 'category',
    'description': 'str',
    'likes': np.int32,
    'comments': np.int32,
    'username': 'category',
    'bio': 'str',
    'followers': np.int32,
    'following': np.int32,
    'num_posts': np.int32,
    'lang': 'category',
    'category': 'category'
}

start_time = time.time()
ddf_opt = dd.read_parquet('*.parquet', dtype=dtype_opt)
ddf_opt['date'] = dd.to_datetime(ddf_opt['date'])
ddf_opt.compute()
end_time = time.time()
print(f"Czas: {end_time - start_time} sekund")

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\io\parquet\core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


Czas: 42.439425230026245 sekund


In [41]:
start_time = time.time()
top10_opt = ddf_opt.groupby('username')['likes'].sum().nlargest(10).compute()
end_time = time.time()
print(f"Czas: {end_time - start_time} sekund")

Czas: 32.59843158721924 sekund


In [42]:
start_time = time.time()
polrocze_opt = ddf_opt[ddf_opt['date'].between('2019-01-01', '2019-06-30')].compute()
end_time = time.time()
print(f"Czas: {end_time - start_time} sekund")

Czas: 43.91252517700195 sekund
